# Kalman Filter
> Implementation of Kalman filters using pytorch and parameter optimizations with gradient descend

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
#| default_exp kalman.filter

In [ ]:
#| export
from fastcore.test import *
from fastcore.basics import patch
from meteo_imp.utils import *
from meteo_imp.data_preparation import MeteoDataTest
import pykalman
from typing import *

import numpy as np
import pandas as pd
import torch
from torch import Tensor
from torch.distributions import MultivariateNormal

## Introduction

## Utils

In [ ]:
#| exporti
def array1d(X):
    """Returns at least 1-d array with data from X"""
    return torch.atleast_1d(torch.as_tensor(X))

def array2d(X):
    """Returns at least 2-d array with data from X"""
    return torch.atleast_2d(torch.as_tensor(X))


In [ ]:
#| export
def _determine_dimensionality(variables, default):
    """Derive the dimensionality of the state space

    Parameters
    ----------
    variables : list of ({None, array}, conversion function, index)
        variables, functions to convert them to arrays, and indices in those
        arrays to derive dimensionality from.
        
    Returns
    -------
    dim : int
        dimensionality of state space as derived from variables or default.
    """
    # gather possible values based on the variables
    candidates = []
    for (v, converter, idx) in variables:
        if v is not None:
            v = converter(v)
            candidates.append(v.shape[idx])
    
     # also use the manually specified default
    if default is not None:
        candidates.append(default)
    
    # ensure consistency of all derived values
    if len(candidates) == 0:
        return 1
    else:
        if not torch.all(torch.tensor(candidates) == candidates[0]):
            raise ValueError(
                "The shape of all " +
                "parameters is not consistent.  " +
                "Please re-check their values."
            )
        return candidates[0]


def _last_dims(X: Tensor, t: int, ndims: int=2):
    """Extract the final dimensions of `X`

    Extract the final `ndim` dimensions at index `t` if `X` has >= `ndim` + 1
    dimensions, otherwise return `X`.

    Parameters
    ----------
    X : Tensor with at least dimension `ndims`
    t : int
        index to use for the `ndims` + 1th dimension
    ndims : int, optional
        number of dimensions in the array desired

    Returns
    -------
    Y : array with dimension `ndims`
        the final `ndims` dimensions indexed by `t`
    """
    if len(X.shape) == ndims + 1:
        return X[t]
    elif len(X.shape) == ndims:
        return X
    else:
        raise ValueError(("X only has %d dimensions when %d" +
                " or more are required") % (len(X.shape), ndims))

In [ ]:
#| export
def _stack_detach(l: Collection[Tensor]):
    return torch.stack(list(map(lambda x: x.detach(), l)))

### Positive Definite Constraint

transform any matrix $A$ into a positive definite matrix ($PD$) using the following formula

$PD = A A^T + aI$

where $AA^T$ ensures that A is positive semi-definite and adding a small positive number of the diagonal $aI$ ensures that is definite 

for details see https://stackoverflow.com/a/58192215/8555045

add also inverse transformation using cholesky decomposition


API inspired by gpytorch constraints

In [ ]:
#| export
def is_pos_semidef(cov):
    return torch.distributions.constraints.positive_semidefinite.check(cov).item()

In [ ]:
#| export
def check_is_pos_semidef(cov):
    if not is_pos_semidef(cov):
        raise ValueError(f"Not positive semi definite {cov}")

In [ ]:
#| export
class PosDef():
    """ Positive Definite Constraint for PyTorch parameters"""
    def __init__(self, a=1e-7):
        self.a = a
    def transform(self,
                  raw # square matrix that will be 
                 ):
        "transform any square matrix into a positive definite one"
        semi_pos = raw @ raw.T
        check_is_pos_semidef(semi_pos)
        return semi_pos + (self.a * torch.eye(raw.shape[0]))
    
    def inverse_transform(self,
                          value # a positive definite matrix
                         ):
        "tranform positive definite matrix into a raw matrix"
        return torch.linalg.cholesky(value)

In [ ]:
A = torch.rand(3,3)
constraint = PosDef()

posdef = constraint.transform(A)

In [ ]:
A

tensor([[0.7483, 0.9382, 0.1555],
        [0.9651, 0.8235, 0.8243],
        [0.9456, 0.2654, 0.9932]])

In [ ]:
posdef

tensor([[1.4644, 1.6230, 1.1111],
        [1.6230, 2.2890, 1.9498],
        [1.1111, 1.9498, 1.9511]])

In [ ]:
test_eq(torch.distributions.constraints.positive_definite.check(posdef), True)

In [ ]:
constraint.inverse_transform(posdef)

tensor([[1.2101, 0.0000, 0.0000],
        [1.3412, 0.7001, 0.0000],
        [0.9182, 1.0261, 0.2349]])

In [ ]:
test_close(posdef, constraint.transform(constraint.inverse_transform(posdef)))

In [ ]:
torch.set_printoptions(precision=10)

In [ ]:
posdef

tensor([[1.4643729925, 1.6229895353, 1.1110808849],
        [1.6229895353, 2.2889847755, 1.9498369694],
        [1.1110808849, 1.9498369694, 1.9510511160]])

In [ ]:
torch.isclose(posdef, posdef.mT, atol=1e-6).all(-2).all(-1)

tensor(True)

In [ ]:
#| export
def is_symmetric(value, atol=1e-5):
    return torch.isclose(value, value.mT, atol=atol).all().item()

In [ ]:
is_symmetric(A)

False

In [ ]:
#| export
def symmetric_upto(value, start=-8):
    for exp in torch.arange(start, 3):
        if is_symmetric(value, atol=10**exp):
            return exp
    return exp

In [ ]:
symmetric_upto(posdef)

tensor(-8)

In [ ]:
symmetric_upto(A)

tensor(0)

In [ ]:
#| export
def is_posdef(cov):
    return torch.distributions.constraints.positive_definite.check(cov).item()

In [ ]:
is_pos_semidef(posdef)

True

In [ ]:
is_posdef(A)

False

In [ ]:
is_posdef(posdef)

True

In [ ]:
torch.linalg.eigvalsh(A)

tensor([-0.4108574390,  0.6451910734,  2.3305833340])

In [ ]:
torch.linalg.eigvalsh(posdef)

tensor([0.0135278013, 0.5718843341, 5.1189956665])

check if it is pos definite using eigenvalues

In [ ]:
MultivariateNormal(torch.stack([torch.ones(3)]*2), torch.stack([posdef, torch.eye(3)]))

MultivariateNormal(loc: torch.Size([2, 3]), covariance_matrix: torch.Size([2, 3, 3]))

In [ ]:
#| export
def is_posdef2(cov):
    eigv = torch.linalg.eigvalsh(cov)
    if (eigv < 0).any():
        return False, eigv
    return True, eigv

In [ ]:
is_posdef2(posdef)

(True, tensor([0.0135278013, 0.5718843341, 5.1189956665]))

In [ ]:
is_posdef2(A)

(False, tensor([-0.4108574390,  0.6451910734,  2.3305833340]))

In [ ]:
#| export
from warnings import warn

In [ ]:
mask = torch.tril(torch.ones(3,3, dtype=torch.bool)).T
mask

tensor([[ True,  True,  True],
        [False,  True,  True],
        [False, False,  True]])

In [ ]:
A[mask]

tensor([0.7482852936, 0.9382153153, 0.1555444002, 0.8234661222, 0.8242675662,
        0.9931656122])

In [ ]:
A[mask] = torch.tril(A)[mask]

In [ ]:
#| export
def make_symmetric(value):
    "drops upper half to make matrix symmetric"
    value = value.clone()
    mask = torch.tril(torch.ones_like(value, dtype=torch.bool))
    value[mask.T] = torch.tril(value)[mask]
    return value

In [ ]:
make_symmetric(A)

tensor([[0.7482852936, 0.9651275277, 0.8234661222],
        [0.9651275277, 0.9456449747, 0.2653839588],
        [0.9456449747, 0.2653839588, 0.9931656122]])

In [ ]:
is_symmetric(make_symmetric(A))

False

In [ ]:
#| export
import pandas as pd

In [ ]:
posdef_log = pd.DataFrame() # None # make it None to disable logging

#### Check pos def

In [ ]:
#| export
def check_posdef(value, name = '', error=True, **extra_args):
    value = value.clone()
    sym_upto = symmetric_upto(value).numpy()
    
    is_pd_eigv, eigv = is_posdef2(value)
    is_pd_chol = torch.linalg.cholesky_ex(value).info.eq(0).all().item()
    is_sym = is_symmetric(value)
  
    # sym = make_symmetric(value)
    # is_pd_forced, _ = is_posdef2(sym)
    
    
    
    info = pd.DataFrame({
        'is_pd_eigv': is_pd_eigv,
        'is_pd_chol': is_pd_chol,
        'is_sym': is_sym,
        'sym_upto': sym_upto,
        # 'force_sym_posdef': is_pd_forced,
        'eigv': [eigv.detach().numpy()],
        'matrix': [value.detach().numpy()],
        'name': name,
        **extra_args
    })
    
    global posdef_log
    if posdef_log is not None:
        posdef_log = pd.concat([posdef_log, info])
    
    if not is_pd_eigv or not is_pd_chol:
        if error:
            warn("Matrix is not positive definite")
    return info

In [ ]:
check_posdef(A, 'A').dtypes

/tmp/ipykernel_209421/527603031.py:33: UserWarning: Matrix is not positive definite
  warn("Matrix is not positive definite")


is_pd_eigv      bool
is_pd_chol      bool
is_sym          bool
sym_upto       int64
eigv          object
matrix        object
name          object
dtype: object

In [ ]:
A

tensor([[0.7482852936, 0.0000000000, 0.0000000000],
        [0.9651275277, 0.8234661222, 0.0000000000],
        [0.9456449747, 0.2653839588, 0.9931656122]])

In [ ]:
posdef_log.dtypes

is_pd_eigv      bool
is_pd_chol      bool
is_sym          bool
sym_upto       int64
eigv          object
matrix        object
name          object
dtype: object

In [ ]:
symmetric_upto(A)

tensor(0)

In [ ]:
check_posdef(A)

/tmp/ipykernel_209421/527603031.py:33: UserWarning: Matrix is not positive definite
  warn("Matrix is not positive definite")


,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix,name
0,False,False,False,0,"[-0.41085744, 0.6451911, 2.3305833]","[[0.7482853, 0.0, 0.0], [0.9651275, 0.8234661,...",


In [ ]:
check_posdef(posdef)

,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix,name
0,True,True,True,-8,"[0.013527801, 0.57188433, 5.1189957]","[[1.464373, 1.6229895, 1.1110809], [1.6229895,...",


In [ ]:
#| export
posdef_log = None # by default disable logging for exported module 

In [ ]:
#| export
def check_posdef4(value, error=True):
    msg = ""
    is_pd_eigv, eigv = is_posdef2(value)
    is_pd_chol = is_posdef(value)
    if not is_pd_eigv or not is_pd_chol:
        is_sym = is_symmetric(value)
        if not is_sym:
            msg += f"Not symmetric, symmetric up to 1e{symmetric_upto(value)}\n"
            msg += f"Not pos definite with eigv {eigv} \n"
        # try to make it symmetric and try again to see if it's posdef
            sym = make_symmetric(value)
            is_pd, eigv = is_posdef2(sym)
            add_msg = "forced symmetric is posdef" if is_pd else "force symmetric is not posdef"
            msg += add_msg
        
    if msg != "":
        msg += str(value)
        warn(msg)

In [ ]:
def check_posdef3(cov, error=True):
    if not is_posdef(cov):
        is_pd, eigv = is_posdef2(cov)
        if is_pd:
            print(f"actually pos definite {eigv} \n{cov}")
        else:
            msg = f"not pos definite {eigv} \n {cov}"
            if error:
                raise ValueError(msg)
            else:
                print(msg)

In [ ]:
def check_posdef2(cov):
    errors = []
    for i in range(10):
        if not is_posdef(cov):
            errors.append(i)
    if len(errors) > 0:
        print(f"at iterations {errors} not positive definite matrix. \n{cov}")

## KalmanFilter

The Kalman Filter is an algorithm designed to estimate $P(x_t | y_{0:t})$.  As all state transitions and obss are linear with Gaussian distributed noise, these distributions can be represented exactly as Gaussian distributions with mean `filt_state_means[t]` and covs `filt_state_covs[t]`.
Similarly, the Kalman Smoother is an algorithm designed to estimate $P(x_t | y_{0:T-1})$



### Main class

In [ ]:
#| export
class KalmanFilter(torch.nn.Module):
    """Kalman Filter and Kalman Smoother using PyTorch"""
    def __init__(self,
            transition_matrices: Tensor=None, # [n_timesteps-1, n_dim_state, n_dim_state] or [n_dim_state,n_dim_state] 
               # Also known as $A$ 
               # state transition matrix between times t and t+1 for t in [0...n_timesteps-2]
            obs_matrices: Tensor=None, # [n_timesteps, n_dim_obs, n_dim_state] or [n_dim_obs, n_dim_state]
                # Also known as $H$
                # obs matrix for times [0...n_timesteps-1]
            transition_cov: Tensor=None, # [n_dim_state, n_dim_state] 
                 # Also known as $Q$
                 # state transition cov matrix for times [0...n_timesteps-2]
            obs_cov: Tensor=None, # [n_dim_obs, n_dim_obs]
                 # Also known as $R$
                 # obs cov matrix for times [0...n_timesteps-1]
            transition_offsets: Tensor=None, # [n_timesteps-1, n_dim_state] or [n_dim_state]
                 # Also known as $b$
                 # state offsets for times [0...n_timesteps-2]
            obs_offsets: Tensor=None, # [n_timesteps, n_dim_obs] or [n_dim_obs]
                 # Also known as $d$
            initial_state_mean: Tensor=None, # [n_dim_state]
                 # Also known as $\mu_0"$
            initial_state_cov: Tensor=None, # [n_dim_state, n_dim_state]
                 # Also known as $\Sigma_0$
            n_dim_state: int = None, # Number of dimensions for state - defaults to 1 if cannot be infered from parameters
            n_dim_obs: int = None # Number of dimensions for observations - defaults to 1 if cannot be infered from parameters
                ):
        """Initialize Kalman Filter"""
        
        super().__init__()
        
        # determine size of state space and check parameters are consistent
        n_dim_state = _determine_dimensionality(
            [(transition_matrices, array2d, -2),
             (transition_offsets, array1d, -1),
             (transition_cov, array2d, -2),
             (initial_state_mean, array1d, -1),
             (initial_state_cov, array2d, -2),
             (obs_matrices, array2d, -1)],
            n_dim_state
        )
        n_dim_obs = _determine_dimensionality(
            [(obs_matrices, array2d, -2),
             (obs_offsets, array1d, -1),
             (obs_cov, array2d, -2)],
            n_dim_obs
        )
        
        self.n_dim_obs = n_dim_obs
        self.n_dim_state = n_dim_state
        
        params = {
        # name                 value         default_value                              converter constraint train
        'transition_matrices': [transition_matrices, torch.eye(n_dim_state),            array2d, None    , True],
        'transition_offsets':  [transition_offsets,  torch.zeros(n_dim_state),          array1d, None    , True],
        'transition_cov':      [transition_cov,      torch.eye(n_dim_state),            array2d, PosDef(), True],
        'obs_matrices':        [obs_matrices,        torch.eye(n_dim_obs, n_dim_state), array2d, None    , True],
        'obs_offsets':         [obs_offsets,         torch.zeros(n_dim_obs),            array1d, None    , True],
        'obs_cov':             [obs_cov,             torch.eye(n_dim_obs),              array2d, PosDef(), True],
        'initial_state_mean':  [initial_state_mean,  torch.zeros(n_dim_state),          array1d, None    , True],
        'initial_state_cov':   [initial_state_cov,   torch.eye(n_dim_state),            array2d, PosDef(), True],
        }
        
        self._init_params(params)
        
        self.check_args = None
        
    def _init_params(self, params):
        for name, (value, default, converter, constraint, train) in params.items():
            value = value if value is not None else default
            value = converter(value) 
            if constraint is not None:
                name, value = self._init_constraint(name, value, constraint)
            self._init_param(name, value, train)    
    
    def _init_param(self, param_name, value, train):
        self.register_parameter(param_name, torch.nn.Parameter(value, requires_grad=train))
    
    ## Constraints ----
    def _init_constraint(self, param_name, value, constraint):
        name = param_name + "_raw"
        value = constraint.inverse_transform(value)
        setattr(self, param_name + "_constraint", constraint)
        return name, value
    
    def _get_constraint(self, param_name):
        constraint = getattr(self, param_name + "_constraint")
        raw_value = getattr(self, param_name + "_raw")
        return constraint.transform(raw_value)
    
    def _set_constraint(self, param_name, value, train=True):
        constraint = getattr(self, param_name + "_constraint")
        raw_value = constraint.inverse_transform(value)
        self._init_param(param_name + "_raw", raw_value, train)
        

    
    ### convenience functions to get and set parameters that have a constraint
    
    @property
    def transition_cov(self):
        return self._get_constraint('transition_cov')
    
    @transition_cov.setter
    def transition_cov(self, value):
        return self._set_constraint('transition_cov', value)
    
    @property
    def obs_cov(self):
        return self._get_constraint('obs_cov')
    
    @obs_cov.setter
    def obs_cov(self, value):
        return self._set_constraint('obs_cov', value)
    
    @property
    def initial_state_cov(self):
        return self._get_constraint('initial_state_cov')
    
    @initial_state_cov.setter
    def initial_state_cov(self, value):
        return self._set_constraint('initial_state_cov', value)
    
    
    ### -----
    
    def _parse_obs(self, obs, mask=None):
        """Safely convert observations to their expected format"""
        obs = torch.atleast_2d(obs)
        if obs.shape[0] == 1 and obs.shape[1] > 1:
            obs = obs.T
        if mask is None: mask = ~torch.isnan(obs).any(axis=1)
        return obs, mask
    
    def __repr__(self):
        return f"""Kalman Filter
        N dim obs: {self.n_dim_obs}, N dim state: {self.n_dim_state}"""

In [ ]:
k = KalmanFilter()

In [ ]:
k.obs_cov

tensor([[1.0000001192]], grad_fn=<AddBackward0>)

In [ ]:
k.initial_state_cov

tensor([[1.0000001192]], grad_fn=<AddBackward0>)

In [ ]:
k.initial_state_cov = array2d(torch.tensor(3.))

In [ ]:
k.initial_state_cov

tensor([[3.]], grad_fn=<AddBackward0>)

In [ ]:
k.initial_state_cov_raw

Parameter containing:
tensor([[1.7320507765]], requires_grad=True)

In [ ]:
list(k.named_parameters())

[('transition_matrices',
  Parameter containing:
  tensor([[1.]], requires_grad=True)),
 ('transition_offsets',
  Parameter containing:
  tensor([0.], requires_grad=True)),
 ('transition_cov_raw',
  Parameter containing:
  tensor([[1.]], requires_grad=True)),
 ('obs_matrices',
  Parameter containing:
  tensor([[1.]], requires_grad=True)),
 ('obs_offsets',
  Parameter containing:
  tensor([0.], requires_grad=True)),
 ('obs_cov_raw',
  Parameter containing:
  tensor([[1.]], requires_grad=True)),
 ('initial_state_mean',
  Parameter containing:
  tensor([0.], requires_grad=True)),
 ('initial_state_cov_raw',
  Parameter containing:
  tensor([[1.7320507765]], requires_grad=True))]

In [ ]:
k = KalmanFilter()
# pykalman reference implementation
pyk = pykalman.KalmanFilter()

In [ ]:
X = torch.tensor([1,2,3])
nX = X.numpy()
X

tensor([1, 2, 3])

2 dimensional dobs

In [ ]:
X2 = torch.tensor([[i, 10. * i] for i in range(1,5)])
nX2 = X2.numpy()

In [ ]:
k2 = KalmanFilter(transition_matrices = torch.eye(2), obs_matrices=torch.eye(2))

pyk2 = pykalman.KalmanFilter(n_dim_obs = 2, n_dim_state=2)

In [ ]:
X2

tensor([[ 1., 10.],
        [ 2., 20.],
        [ 3., 30.],
        [ 4., 40.]])

In [ ]:
obs_mask = torch.ones_like(X2, dtype=torch.bool)
obs_mask[0, 1] = False # only one value missing
obs_mask[2, :] = False # whole row missing
# Xm X missing
X2m = X2.clone()
X2m[~obs_mask] = torch.nan
# nXm Numpy X missing
nX2m = np.ma.masked_array(X2.numpy(), mask = ~obs_mask.numpy())

In [ ]:
obs_mask

tensor([[ True, False],
        [ True,  True],
        [False, False],
        [ True,  True]])

In [ ]:
X2m

tensor([[ 1., nan],
        [ 2., 20.],
        [nan, nan],
        [ 4., 40.]])

In [ ]:
nX2m

masked_array(
  data=[[1.0, --],
        [2.0, 20.0],
        [--, --],
        [4.0, 40.0]],
  mask=[[False,  True],
        [False, False],
        [ True,  True],
        [False, False]],
  fill_value=1e+20,
  dtype=float32)

### Tester

In [ ]:
#| export
from fastcore.basics import *

In [ ]:
#| export
to_posdef = PosDef(a=1e-5).transform

class KalmanFilterTester():
    
    torch2pyk = {
        'transition_matrices': 'transition_matrices',
        'transition_offsets':  'transition_offsets',        
        'transition_cov':      'transition_covariance',        
        'obs_matrices':        'observation_matrices',
        'obs_offsets':         'observation_offsets',          
        'obs_cov':             'observation_covariance',            
        'initial_state_mean':  'initial_state_mean',        
        'initial_state_cov':   'initial_state_covariance',
    }
    
    def __init__(self,
                 n_dim_state = 3,
                n_dim_obs = 3,
                n_obs = 10,
                p_missing = .3,
                seed=42,
                dtype=torch.float32,
                nan_mask = True
                ):
        store_attr(but='seed')
        reset_seed(seed)
        
        self.random_init()
    
    def random_init(self):
        self.params = {
            'transition_matrices': torch.rand(self.n_dim_state, self.n_dim_state, dtype=self.dtype),
            'transition_offsets':  torch.rand(self.n_dim_state, dtype=self.dtype),        
            'transition_cov':      to_posdef(torch.rand(self.n_dim_state, self.n_dim_state, dtype=self.dtype)),        
            'obs_matrices':        torch.rand(self.n_dim_obs, self.n_dim_state, dtype=self.dtype),
            'obs_offsets':         torch.rand(self.n_dim_obs, dtype=self.dtype),          
            'obs_cov':             to_posdef(torch.rand(self.n_dim_obs, self.n_dim_obs, dtype=self.dtype)),            
            'initial_state_mean':  torch.rand(self.n_dim_state, dtype=self.dtype),        
            'initial_state_cov':   to_posdef(torch.rand(self.n_dim_state, self.n_dim_state, dtype=self.dtype)),
        }
        self.params_pyk = {self.torch2pyk[name]: param.numpy() for name, param in self.params.items()}
        
        self.filter = KalmanFilter(**self.params)
        self.filter_pyk = pykalman.standard.KalmanFilter(**self.params_pyk)
        
        
        self.data = torch.rand(self.n_obs, self.n_dim_obs, dtype=self.dtype)
        self.mask = torch.rand(self.n_obs, self.n_dim_obs) > self.p_missing
        if self.nan_mask: self.data[~self.mask] = torch.nan # ensure that the data cannot be used
        self.mask_torch = self.mask.all(1)
        self.data_pyk = np.ma.masked_array(self.data.numpy(), mask = ~self.mask.numpy()) 
    
    

In [ ]:
tst = KalmanFilterTester()

In [ ]:
tst64 = KalmanFilterTester(dtype=torch.float64)

In [ ]:
test_close(tst.params.values(), tst.params_pyk.values())

### Filter

#### Filter predict

In [ ]:
#| export
from datetime import datetime
def _filter_predict(transition_matrix, transition_cov,
                    transition_offset, current_state_mean,
                    current_state_cov, check_args=None):
    r"""Calculate the mean and cov of :math:`P(x_{t+1} | z_{0:t})`

    Using the mean and cov of :math:`P(x_t | z_{0:t})`, calculate the
    mean and cov of :math:`P(x_{t+1} | z_{0:t})`.

    Parameters
    ----------
    transition_matrix : [n_dim_state, n_dim_state} array
        state transition matrix from time t to t+1
    transition_cov : [n_dim_state, n_dim_state] array
        cov matrix for state transition from time t to t+1
    transition_offset : [n_dim_state] array
        offset for state transition from time t to t+1
    current_state_mean: [n_dim_state] array
        mean of state at time t given obss from times
        [0...t]
    current_state_cov: [n_dim_state, n_dim_state] array
        cov of state at time t given obss from times
        [0...t]

    Returns
    -------
    pred_state_mean : [n_dim_state] array
        mean of state at time t+1 given obss from times [0...t]
    pred_state_cov : [n_dim_state, n_dim_state] array
        cov of state at time t+1 given obss from times
        [0...t]
    """
    pred_state_mean = transition_matrix @ current_state_mean + transition_offset
    pred_state_cov =  transition_matrix @ current_state_cov @ transition_matrix.T + transition_cov

    if check_args is not None: check_posdef(pred_state_cov, 'filter_predict', **check_args)
    
    return (pred_state_mean, pred_state_cov)

In [ ]:
def _filter_predict2(transition_matrix, transition_covariance,
                    transition_offset, current_state_mean,
                    current_state_covariance):
    predicted_state_mean = (
        torch.matmul(transition_matrix, current_state_mean)
        + transition_offset
    )
    predicted_state_covariance = (
        torch.matmul(transition_matrix,
               torch.matmul(current_state_covariance,
                      transition_matrix.T))
        + transition_covariance
    )

    return (predicted_state_mean, predicted_state_covariance)

In [ ]:
(
    (tst.params['transition_matrices'] @ tst.params['initial_state_cov'] ) -
    torch.matmul(tst.params['transition_matrices'], tst.params['initial_state_cov'])
)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
(
    (tst.params['transition_matrices'] @ tst.params['initial_state_cov']  @ tst.params['transition_matrices'].T) -
    torch.matmul(tst.params['transition_matrices'], torch.matmul(tst.params['initial_state_cov'], tst.params['transition_matrices'].T))
)

tensor([[ 0.0000000000e+00, -2.3841857910e-07, -2.3841857910e-07],
        [ 0.0000000000e+00,  0.0000000000e+00,  0.0000000000e+00],
        [ 0.0000000000e+00,  0.0000000000e+00,  0.0000000000e+00]])

In [ ]:
(
    (tst.params['transition_matrices'] @ tst.params['initial_state_cov']  @ tst.params['transition_matrices'].T + tst.params['transition_cov']) -
    (torch.matmul(tst.params['transition_matrices'], torch.matmul(tst.params['initial_state_cov'], tst.params['transition_matrices'].T)) + tst.params['transition_cov'])
)

tensor([[ 0.0000000000e+00,  0.0000000000e+00, -2.3841857910e-07],
        [ 0.0000000000e+00,  0.0000000000e+00,  0.0000000000e+00],
        [ 0.0000000000e+00,  0.0000000000e+00,  0.0000000000e+00]])

In [ ]:
trans_m = np.eye(2)
trans_cov = np.eye(2)
trans_off = np.zeros((2,2))
curr_mean = np.ones((2,1))
curr_cov = np.zeros((2,2))

In [ ]:
_filter_predict(torch.tensor(trans_m) , torch.tensor(trans_cov), torch.tensor(trans_off), torch.tensor(curr_mean), torch.tensor(curr_cov))

(tensor([[1., 1.],
         [1., 1.]], dtype=torch.float64),
 tensor([[1., 0.],
         [0., 1.]], dtype=torch.float64))

In [ ]:
test_close(
    pykalman.standard._filter_predict(trans_m , trans_cov, trans_off, curr_mean, curr_cov),
    _filter_predict(torch.tensor(trans_m) , torch.tensor(trans_cov), torch.tensor(trans_off), torch.tensor(curr_mean), torch.tensor(curr_cov)))

In [ ]:
pred_pyk = pykalman.standard._filter_predict(
   tst.params_pyk['transition_matrices'],
   tst.params_pyk['transition_covariance'],
   tst.params_pyk['transition_offsets'],
   tst.params_pyk['initial_state_mean'],
   tst.params_pyk['initial_state_covariance'],
)

In [ ]:
pred_torch = _filter_predict(
   tst.params['transition_matrices'],
   tst.params['transition_cov'],
   tst.params['transition_offsets'],
   tst.params['initial_state_mean'],
   tst.params['initial_state_cov'],

)

In [ ]:
pred_pyk[0] - pred_torch[0].numpy()

array([0., 0., 0.], dtype=float32)

In [ ]:
test_close(pred_pyk[0], pred_torch[0])

In [ ]:
pred_pyk[1] - pred_torch[1].numpy()

array([[0.0000000e+00, 0.0000000e+00, 2.3841858e-07],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [ ]:
test_close(pred_pyk[1], pred_torch[1])

In [ ]:
pykalman.standard._filter_predict??

Signature:
pykalman.standard._filter_predict(
    transition_matrix,
    transition_covariance,
    transition_offset,
    current_state_mean,
    current_state_covariance,
)
Source:   
def _filter_predict(transition_matrix, transition_covariance,
                    transition_offset, current_state_mean,
                    current_state_covariance):
    r"""Calculate the mean and covariance of :math:`P(x_{t+1} | z_{0:t})`

    Using the mean and covariance of :math:`P(x_t | z_{0:t})`, calculate the
    mean and covariance of :math:`P(x_{t+1} | z_{0:t})`.

    Parameters
    ----------
    transition_matrix : [n_dim_state, n_dim_state} array
        state transition matrix from time t to t+1
    transition_covariance : [n_dim_state, n_dim_state] array
        covariance matrix for state transition from time t to t+1
    transition_offset : [n_dim_state] array
        offset for state transition from time t to t+1
    current_state_mean: [n_dim_state] array
        mean of state at tim

In [ ]:
tst.params_pyk['transition_matrices'].shape

(3, 3)

In [ ]:
test_close(
   pykalman.standard._filter_predict(
       tst.params_pyk['transition_matrices'],
       tst.params_pyk['transition_covariance'],
       tst.params_pyk['transition_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
    ),
    
    _filter_predict2(
       tst.params['transition_matrices'],
       tst.params['transition_cov'],
       tst.params['transition_offsets'],
       tst.params['initial_state_mean'],
       tst.params['initial_state_cov'],
    
    )
)

In [ ]:
test_close(
    
    _filter_predict2(
       tst.params['transition_matrices'],
       tst.params['transition_cov'],
       tst.params['transition_offsets'],
       tst.params['initial_state_mean'],
       tst.params['initial_state_cov'],
    ),
    _filter_predict(
       tst.params['transition_matrices'],
       tst.params['transition_cov'],
       tst.params['transition_offsets'],
       tst.params['initial_state_mean'],
       tst.params['initial_state_cov'],
    )
)

In [ ]:
test_close(
   pykalman.standard._filter_predict(
       tst.params_pyk['transition_matrices'],
       tst.params_pyk['transition_covariance'],
       tst.params_pyk['transition_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
    ),
    
    _filter_predict(
       tst.params['transition_matrices'],
       tst.params['transition_cov'],
       tst.params['transition_offsets'],
       tst.params['initial_state_mean'],
       tst.params['initial_state_cov'],
    
    )
)

The issues seems is was from `check_posdef` but `torch.matmul` and `@` are correctly working as intended

#### Filter correct

In [ ]:
pykalman.standard._filter_correct??

Signature:
pykalman.standard._filter_correct(
    observation_matrix,
    observation_covariance,
    observation_offset,
    predicted_state_mean,
    predicted_state_covariance,
    observation,
)
Source:   
def _filter_correct(observation_matrix, observation_covariance,
                    observation_offset, predicted_state_mean,
                    predicted_state_covariance, observation):
    r"""Correct a predicted state with a Kalman Filter update

    Incorporate observation `observation` from time `t` to turn
    :math:`P(x_t | z_{0:t-1})` into :math:`P(x_t | z_{0:t})`

    Parameters
    ----------
    observation_matrix : [n_dim_obs, n_dim_state] array
        observation matrix for time t
    observation_covariance : [n_dim_obs, n_dim_obs] array
        covariance matrix for observation at time t
    observation_offset : [n_dim_obs] array
        offset for observation at time t
    predicted_state_mean : [n_dim_state] array
        mean of state at time t given observatio

In [ ]:
tst = KalmanFilterTester(p_missing=0, nan_mask=False)

In [ ]:
observation_matrix = tst.params_pyk['observation_matrices']
observation_covariance = tst.params_pyk['observation_covariance']
predicted_state_mean = tst.params_pyk['initial_state_mean']
predicted_state_covariance = tst.params_pyk['initial_state_covariance']
observation_offset = tst.params_pyk['observation_offsets']
observation = tst.data_pyk[0]

In [ ]:
observation

masked_array(data=[0.6440156698226929, 0.7071067690849304,
                   0.6581305861473083],
             mask=[False, False, False],
       fill_value=1e+20,
            dtype=float32)

In [ ]:
obs_matrix = tst.params['obs_matrices']
obs_cov = tst.params['obs_cov']
pred_state_mean = tst.params['initial_state_mean']
pred_state_cov = tst.params['initial_state_cov']
obs_offset = tst.params['obs_offsets']
obs = tst.data[0]

Pred obs mean

In [ ]:
predicted_observation_mean = (
    np.dot(observation_matrix,
           predicted_state_mean)
    + observation_offset
)

In [ ]:
pred_obs_mean = obs_matrix @ pred_state_mean + obs_offset

In [ ]:
pred_obs_mean

tensor([2.2023458481, 0.6882513762, 1.5503764153])

In [ ]:
test_close(pred_obs_mean, predicted_observation_mean)

Pred obs cov

In [ ]:
predicted_observation_covariance = (
    np.dot(observation_matrix,
           np.dot(predicted_state_covariance,
                  observation_matrix.T))
    + observation_covariance
)

In [ ]:
pred_obs_cov = obs_matrix @ pred_state_cov @ obs_matrix.T + obs_cov

In [ ]:
pred_obs_cov

tensor([[2.7632296085, 1.7844555378, 1.2538765669],
        [1.7844555378, 1.7600629330, 1.2308311462],
        [1.2538764477, 1.2308311462, 0.9597316980]])

In [ ]:
test_close(predicted_observation_covariance, pred_obs_cov)

Kalman gain

In [ ]:
kalman_gain = (
    np.dot(predicted_state_covariance,
           np.dot(observation_matrix.T,
                  np.linalg.pinv(predicted_observation_covariance)))
)

In [ ]:
kalman_gain_torch = pred_state_cov @ obs_matrix.T @ torch.inverse(pred_obs_cov)

In [ ]:
kalman_gain_torch

tensor([[ 0.7220908999, -1.0673114061,  0.8582634926],
        [ 0.2587437928, -1.5012981892,  2.0114912987],
        [ 0.7195257545, -0.0971620083, -0.5134804249]])

In [ ]:
test_close(kalman_gain, kalman_gain_torch)

corr state mean

In [ ]:
corrected_state_mean = (
    predicted_state_mean
    + np.dot(kalman_gain, observation - predicted_observation_mean))

In [ ]:
corr_state_mean = pred_state_mean + kalman_gain_torch @ (obs - pred_obs_mean)

In [ ]:
corr_state_mean

tensor([-1.2007339001, -1.2798495293,  0.1240897775])

In [ ]:
test_close(corrected_state_mean, corr_state_mean)

corr state cov

In [ ]:
corrected_state_covariance = (
    predicted_state_covariance
    - np.dot(kalman_gain,
             np.dot(observation_matrix,
                    predicted_state_covariance))
)

In [ ]:
corr_state_cov = pred_state_cov - kalman_gain_torch @ obs_matrix @ pred_state_cov

In [ ]:
corr_state_cov

tensor([[ 0.3472324014, -0.0682768822,  0.1338058710],
        [-0.0682771206,  0.0863963962,  0.0947692692],
        [ 0.1338058114,  0.0947694778,  0.2892869115]])

In [ ]:
test_close(corrected_state_covariance, corr_state_cov)

In [ ]:
def print_info(x, name):
    print(f"{name} - shape: {x.shape}, type {x.dtype}, mean {x.mean()}")

In [ ]:
#| export
def _filter_correct(obs_matrix, obs_cov,
                    obs_offset, pred_state_mean,
                    pred_state_cov, obs, mask, check_args=None):
    r"""Correct a pred state with a Kalman Filter update

    Incorporate obs `obs` from time `t` to turn
    :math:`P(x_t | z_{0:t-1})` into :math:`P(x_t | z_{0:t})`

    Parameters
    ----------
    obs_matrix : [n_dim_obs, n_dim_state] array
        obs matrix for time t
    obs_cov : [n_dim_obs, n_dim_obs] array
        cov matrix for obs at time t
    obs_offset : [n_dim_obs] array
        offset for obs at time t
    pred_state_mean : [n_dim_state] array
        mean of state at time t given obss from times
        [0...t-1]
    pred_state_cov : [n_dim_state, n_dim_state] array
        cov of state at time t given obss from times
        [0...t-1]
    obs : [n_dim_obs] array
        obs at time t.  If `obs` is a masked array and any of
        its values are masked, the obs will be ignored.

    Returns
    -------
    kalman_gain : [n_dim_state, n_dim_obs] array
        Kalman gain matrix for time t
    corrected_state_mean : [n_dim_state] array
        mean of state at time t given obss from times
        [0...t]
    corrected_state_cov : [n_dim_state, n_dim_state] array
        cov of state at time t given obss from times
        [0...t]
    """
    if mask:
        pred_obs_mean = obs_matrix @ pred_state_mean + obs_offset
        pred_obs_cov = obs_matrix @ pred_state_cov @ obs_matrix.T + obs_cov
        
        kalman_gain = pred_state_cov @ obs_matrix.T @ torch.inverse(pred_obs_cov)

        corrected_state_mean = pred_state_mean + kalman_gain @ (obs - pred_obs_mean)
        corrected_state_cov = pred_state_cov - kalman_gain @ obs_matrix @ pred_state_cov
    else:
        n_dim_state = pred_state_cov.shape[0]
        n_dim_obs = obs_matrix.shape[0]
        kalman_gain = torch.zeros((n_dim_state, n_dim_obs))

        corrected_state_mean = pred_state_mean
        corrected_state_cov = pred_state_cov
        
    if check_args is not None: check_posdef(pred_state_cov, 'filter_correct', **check_args)
    return (kalman_gain, corrected_state_mean,
            corrected_state_cov)

In [ ]:
np.any(np.ma.getmask(observation))

False

In [ ]:
def _pyk_filter_correct(observation_matrix, observation_covariance,
                    observation_offset, predicted_state_mean,
                    predicted_state_covariance, observation):
    if not np.any(np.ma.getmask(observation)):
        predicted_observation_mean = (
            np.dot(observation_matrix,
                   predicted_state_mean)
            + observation_offset
        )
        print_info(predicted_observation_mean, 'pred_obs_mean')
        predicted_observation_covariance = (
            np.dot(observation_matrix,
                   np.dot(predicted_state_covariance,
                          observation_matrix.T))
            + observation_covariance
        )
        print_info(predicted_observation_covariance, 'pred_obs_cov')

        kalman_gain = (
            np.dot(predicted_state_covariance,
                   np.dot(observation_matrix.T,
                          np.linalg.pinv(predicted_observation_covariance)))
        )
        print_info(kalman_gain, 'kalman_gain')

        corrected_state_mean = (
            predicted_state_mean
            + np.dot(kalman_gain, observation - predicted_observation_mean)
        )
        print_info(corrected_state_mean, 'corr_state_mean')
        corrected_state_covariance = (
            predicted_state_covariance
            - np.dot(kalman_gain,
                     np.dot(observation_matrix,
                            predicted_state_covariance))
        )
        print_info(corrected_state_covariance, 'corr_state_cov')
    else:
        n_dim_state = predicted_state_covariance.shape[0]
        n_dim_obs = observation_matrix.shape[0]
        kalman_gain = np.zeros((n_dim_state, n_dim_obs))

        corrected_state_mean = predicted_state_mean
        corrected_state_covariance = predicted_state_covariance

    return (kalman_gain, corrected_state_mean,
            corrected_state_covariance)

In [ ]:
tst = KalmanFilterTester() # need nan

In [ ]:
_filter_correct(
   tst.params['obs_matrices'],
   tst.params['obs_cov'],
   tst.params['obs_offsets'],
   tst.params['initial_state_mean'],
   tst.params['initial_state_cov'],
   tst.data[0],
   tst.mask_torch[0]
)

(tensor([[ 0.7220908999, -1.0673114061,  0.8582634926],
         [ 0.2587437928, -1.5012981892,  2.0114912987],
         [ 0.7195257545, -0.0971620083, -0.5134804249]]),
 tensor([-1.2007339001, -1.2798495293,  0.1240897775]),
 tensor([[ 0.3472324014, -0.0682768822,  0.1338058710],
         [-0.0682771206,  0.0863963962,  0.0947692692],
         [ 0.1338058114,  0.0947694778,  0.2892869115]]))

In [ ]:
tst.data

tensor([[0.6440156698, 0.7071067691, 0.6581305861],
        [         nan,          nan, 0.1447432041],
        [0.5314818621, 0.1587299109, 0.6541759968],
        [0.3278088570,          nan,          nan],
        [0.9146959186, 0.2036490440, 0.2018010020],
        [         nan,          nan, 0.6666255593],
        [0.9811253548,          nan, 0.0040619373],
        [         nan,          nan,          nan],
        [         nan,          nan, 0.2417873144],
        [0.1591441035, 0.7652890682,          nan]])

In [ ]:
tst.mask

tensor([[ True,  True,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True, False, False],
        [ True,  True,  True],
        [False, False,  True],
        [ True, False,  True],
        [False, False, False],
        [False, False,  True],
        [ True,  True, False]])

In [ ]:
tst.mask_torch

tensor([ True, False,  True, False,  True, False, False, False, False, False])

In [ ]:
pykalman.standard._filter_correct(
       tst.params_pyk['observation_matrices'],
       tst.params_pyk['observation_covariance'],
       tst.params_pyk['observation_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
       tst.data_pyk[0]
    )

(array([[ 0.7220905 , -1.0673099 ,  0.8582611 ],
        [ 0.25874323, -1.5012974 ,  2.0114906 ],
        [ 0.7195257 , -0.09716119, -0.51348233]], dtype=float32),
 masked_array(data=[-1.2007310390472412, -1.2798478603363037,
                    0.12409162521362305],
              mask=[False, False, False],
        fill_value=1e+20,
             dtype=float32),
 array([[ 0.34723306, -0.0682762 ,  0.13380629],
        [-0.06827652,  0.08639681,  0.09476982],
        [ 0.13380623,  0.09476998,  0.2892871 ]], dtype=float32))

In [ ]:
_pyk_filter_correct(
       tst.params_pyk['observation_matrices'],
       tst.params_pyk['observation_covariance'],
       tst.params_pyk['observation_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
       tst.data_pyk[0]
    )

pred_obs_mean - shape: (3,), type float32, mean 1.480324625968933
pred_obs_cov - shape: (3, 3), type float32, mean 1.5579278469085693
kalman_gain - shape: (3, 3), type float32, mean 0.1545402556657791
corr_state_mean - shape: (3,), type float32, mean -0.7854975064595541
corr_state_cov - shape: (3, 3), type float32, mean 0.11594609171152115


(array([[ 0.7220905 , -1.0673113 ,  0.85826373],
        [ 0.25874335, -1.5012981 ,  2.0114913 ],
        [ 0.7195256 , -0.09716291, -0.5134798 ]], dtype=float32),
 masked_array(data=[-1.2007334232330322, -1.2798488140106201,
                    0.12408965826034546],
              mask=[False, False, False],
        fill_value=1e+20,
             dtype=float32),
 array([[ 0.34723264, -0.06827685,  0.13380617],
        [-0.06827661,  0.08639663,  0.09476981],
        [ 0.13380617,  0.09476957,  0.28928733]], dtype=float32))

In [ ]:
pykalman.standard._filter_correct(
   tst.params['obs_matrices'].numpy(),
   tst.params['obs_cov'].numpy(),
   tst.params['obs_offsets'].numpy(),
   tst.params['initial_state_mean'].numpy(),
   tst.params['initial_state_cov'].numpy(),
   np.ma.masked_array(tst.data[0], mask=[True, True, True])
)

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([0.7104288, 0.9464111, 0.7890298], dtype=float32),
 array([[1.0486114 , 0.3691155 , 0.7147802 ],
        [0.3691155 , 0.60654867, 0.25865388],
        [0.7147802 , 0.25865388, 0.9363084 ]], dtype=float32))

In [ ]:
_pyk_filter_correct(
   tst.params['obs_matrices'].numpy(),
   tst.params['obs_cov'].numpy(),
   tst.params['obs_offsets'].numpy(),
   tst.params['initial_state_mean'].numpy(),
   tst.params['initial_state_cov'].numpy(),
   np.ma.masked_array(tst.data[0], mask=[True, True, True])
)

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([0.7104288, 0.9464111, 0.7890298], dtype=float32),
 array([[1.0486114 , 0.3691155 , 0.7147802 ],
        [0.3691155 , 0.60654867, 0.25865388],
        [0.7147802 , 0.25865388, 0.9363084 ]], dtype=float32))

In [ ]:
   (pykalman.standard._filter_correct(
       tst.params_pyk['observation_matrices'],
       tst.params_pyk['observation_covariance'],
       tst.params_pyk['observation_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
       tst.data_pyk[0]
    )[0] -
    
    _filter_correct(
       tst.params['obs_matrices'],
       tst.params['obs_cov'],
       tst.params['obs_offsets'],
       tst.params['initial_state_mean'],
       tst.params['initial_state_cov'],
       tst.data[0],
       tst.mask_torch[0]
    )[0].numpy())

array([[-4.1723251e-07,  1.5497208e-06, -2.3841858e-06],
       [-5.6624413e-07,  8.3446503e-07, -7.1525574e-07],
       [-5.9604645e-08,  8.1956387e-07, -1.9073486e-06]], dtype=float32)

In [ ]:
args_np = {
    'obs_m': np.eye(2),
    'obs_cov': np.eye(2),
    'obs_off': np.zeros((2,1)),
    'pred_state_mean': np.ones(2),
    'pred_state_cov': np.eye(2),
    'obs': np.ones((2,1)),
}

args_torch = {k: torch.tensor(v) for k,v in args_np.items()}

In [ ]:
_filter_correct(*args_torch.values(), mask=True)

(tensor([[0.5000000000, 0.0000000000],
         [0.0000000000, 0.5000000000]], dtype=torch.float64),
 tensor([[1., 1.],
         [1., 1.]], dtype=torch.float64),
 tensor([[0.5000000000, 0.0000000000],
         [0.0000000000, 0.5000000000]], dtype=torch.float64))

In [ ]:
test_close(
    pykalman.standard._filter_correct(*args_np.values()),
    _filter_correct(*args_torch.values(), mask=True))

In [ ]:
test_close(
   pykalman.standard._filter_correct(
       tst.params_pyk['observation_matrices'],
       tst.params_pyk['observation_covariance'],
       tst.params_pyk['observation_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
       tst.data_pyk[0]
    ),
    
    _filter_correct(
       tst.params['obs_matrices'],
       tst.params['obs_cov'],
       tst.params['obs_offsets'],
       tst.params['initial_state_mean'],
       tst.params['initial_state_cov'],
       tst.data[0],
       tst.mask_torch[0]
    )
)

trying multiple random values ...

In [ ]:
for _ in range(100):
    tst = KalmanFilterTester(nan_mask=False)
    test_close(
       pykalman.standard._filter_correct(
           tst.params_pyk['observation_matrices'],
           tst.params_pyk['observation_covariance'],
           tst.params_pyk['observation_offsets'],
           tst.params_pyk['initial_state_mean'],
           tst.params_pyk['initial_state_covariance'],
           tst.data_pyk[0]
        ),

        _filter_correct(
           tst.params['obs_matrices'],
           tst.params['obs_cov'],
           tst.params['obs_offsets'],
           tst.params['initial_state_mean'],
           tst.params['initial_state_cov'],
           tst.data[0],
           tst.mask_torch[0]
        )
    )

#### Filter

In [ ]:
#| export
def _filter(transition_matrices, obs_matrices, transition_cov,
            obs_cov, transition_offsets, obs_offsets,
            initial_state_mean, initial_state_cov, obs, obs_mask, check_args={}):
    """Apply the Kalman Filter

    Calculate posterior distribution over hidden states given obss up
    to and including the current time step.

    Parameters
    ----------
    transition_matrices : [n_timesteps-1,n_dim_state,n_dim_state] or
    [n_dim_state,n_dim_state] array-like
        state transition matrices
    obs_matrices : [n_timesteps, n_dim_obs, n_dim_state] or [n_dim_obs, \
    n_dim_state] array-like
        obs matrix
    transition_cov : [n_timesteps-1,n_dim_state,n_dim_state] or
    [n_dim_state,n_dim_state] array-like
        state transition cov matrix
    obs_cov : [n_timesteps, n_dim_obs, n_dim_obs] or [n_dim_obs,
    n_dim_obs] array-like
        obs cov matrix
    transition_offsets : [n_timesteps-1, n_dim_state] or [n_dim_state] \
    array-like
        state offset
    obs_offsets : [n_timesteps, n_dim_obs] or [n_dim_obs] array-like
        obss for times [0...n_timesteps-1]
    initial_state_mean : [n_dim_state] array-like
        mean of initial state distribution
    initial_state_cov : [n_dim_state, n_dim_state] array-like
        cov of initial state distribution
    obss : [n_timesteps, n_dim_obs] array
        obss from times [0...n_timesteps-1].  If `obss` is a
        masked array and any of `obss[t]` is masked, then
        `obss[t]` will be treated as a missing obs.

    Returns
    -------
    pred_state_means : [n_timesteps, n_dim_state] array
        `pred_state_means[t]` = mean of hidden state at time t given
        obss from times [0...t-1]
    pred_state_covs : [n_timesteps, n_dim_state, n_dim_state] array
        `pred_state_covs[t]` = cov of hidden state at time t
        given obss from times [0...t-1]
    kalman_gains : [n_timesteps, n_dim_state] array
        `kalman_gains[t]` = Kalman gain matrix for time t
    filt_state_means : [n_timesteps, n_dim_state] array
        `filt_state_means[t]` = mean of hidden state at time t given
        obss from times [0...t]
    filt_state_covs : [n_timesteps, n_dim_state] array
        `filt_state_covs[t]` = cov of hidden state at time t
        given obss from times [0...t]
    """
    n_timesteps = obs.shape[0]
    n_dim_state = len(initial_state_mean)
    n_dim_obs = obs.shape[1]
    
    # those variables need to be lists and not Tensors,
    # otherwise pytorch tryies to compute the gradient for the whole tensor and it breaks due to the in place operations
    
    pred_state_means = [None for _ in range(n_timesteps)] # torch.zeros((n_timesteps, n_dim_state))
    pred_state_covs = [None for _ in range(n_timesteps)] # torch.zeros(
        #(n_timesteps, n_dim_state, n_dim_state)
    #)
    kalman_gains = [None for _ in range(n_timesteps)]
    filt_state_means = [None for _ in range(n_timesteps)]
    filt_state_covs = [None for _ in range(n_timesteps)]

    for t in range(n_timesteps):
        if t == 0:
            pred_state_means[t] = initial_state_mean
            pred_state_covs[t] = initial_state_cov
        else:
            transition_matrix = _last_dims(transition_matrices, t - 1)
            transition_cov = _last_dims(transition_cov, t - 1)
            transition_offset = _last_dims(transition_offsets, t - 1, ndims=1)
            pred_state_means[t], pred_state_covs[t] = (
                _filter_predict(
                    transition_matrix,
                    transition_cov,
                    transition_offset,
                    filt_state_means[t - 1],
                    filt_state_covs[t - 1],
                    check_args = {'t': t, **check_args} if check_args is not None else None
                )
            )

        obs_matrix = _last_dims(obs_matrices, t)
        obs_cov = _last_dims(obs_cov, t)
        obs_offset = _last_dims(obs_offsets, t, ndims=1)
        (kalman_gains[t], filt_state_means[t],
         filt_state_covs[t]) = (
            _filter_correct(obs_matrix,
                obs_cov,
                obs_offset,
                pred_state_means[t],
                pred_state_covs[t],
                obs[t],
                obs_mask[t],
                check_args = {'t': t, **check_args} if check_args is not None else None
            )
        )

    return (pred_state_means, pred_state_covs, filt_state_means,
            filt_state_covs)

In [ ]:
def test_filter():
    pred_s, pred_s_cov, kal, filt_s, filt_s_cov =  pykalman.standard._filter(
       tst.params_pyk['transition_matrices'],
       tst.params_pyk['observation_matrices'],
       tst.params_pyk['transition_covariance'],
       tst.params_pyk['observation_covariance'],
       tst.params_pyk['transition_offsets'],
       tst.params_pyk['observation_offsets'],
       tst.params_pyk['initial_state_mean'],
       tst.params_pyk['initial_state_covariance'],
       tst.data_pyk[0:1]
    )
    
    pyk = (pred_s, pred_s_cov, filt_s, filt_s_cov,) # results without kalman gain
    
    filter_torch = tuple(map(
        torch.vstack, # need to convert lists to tensor
          _filter(
           tst.params['transition_matrices'],
           tst.params['obs_matrices'],
           tst.params['transition_cov'],
           tst.params['obs_cov'],
           tst.params['transition_offsets'],
           tst.params['obs_offsets'],
           tst.params['initial_state_mean'],
           tst.params['initial_state_cov'],
           tst.data[0:1],
           tst.mask_torch[0:1]
    )))
    
    test_close(pyk, filter_torch)

In [ ]:
test_filter()

##### Old testing

In [ ]:
args_filt_np = {
    'trans_m': np.eye(2),
    'obs_m': np.eye(2),
    'trans_cov': np.eye(2),
    'obs_cov': np.eye(2),
    'trans_off': np.zeros((2,1)),
    'obs_off': np.zeros((3,2)),
    'init_state_mean': np.ones(2),
    'init_state_cov': np.eye(2),
    'obs': np.ones((3, 2)),
}

obs_mask = np.ones(3)

args_filt_torch = {k: torch.tensor(v, dtype = torch.float32) for k,v in args_filt_np.items()}

In [ ]:
_filter(*args_filt_torch.values(), obs_mask=obs_mask)

([tensor([1., 1.]), tensor([1., 1.]), tensor([1., 1.])],
 [tensor([[1., 0.],
          [0., 1.]]),
  tensor([[1.5000000000, 0.0000000000],
          [0.0000000000, 1.5000000000]]),
  tensor([[1.5999999046, 0.0000000000],
          [0.0000000000, 1.5999999046]])],
 [tensor([1., 1.]), tensor([1., 1.]), tensor([1., 1.])],
 [tensor([[0.5000000000, 0.0000000000],
          [0.0000000000, 0.5000000000]]),
  tensor([[0.5999999642, 0.0000000000],
          [0.0000000000, 0.5999999642]]),
  tensor([[0.6153846383, 0.0000000000],
          [0.0000000000, 0.6153846383]])])

In [ ]:
filt_pyk = list(pykalman.standard._filter(*args_filt_np.values()))
del filt_pyk[2] # remove kalman gain that is not returned py _filter

filt =  list(map(lambda x: torch.stack(x), _filter(*args_filt_torch.values(), obs_mask=obs_mask)))

test_close( filt_pyk, filt)

missing data

In [ ]:
obs_mask = np.array([True, False, True])
args_filt_np['obs'] = np.ma.masked_array(args_filt_np['obs'], mask = np.vstack([~obs_mask] * 2).T)

In [ ]:
args_filt_np['obs']

masked_array(
  data=[[1.0, 1.0],
        [--, --],
        [1.0, 1.0]],
  mask=[[False, False],
        [ True,  True],
        [False, False]],
  fill_value=1e+20)

In [ ]:
filt_pyk = list(pykalman.standard._filter(*args_filt_np.values()))
del filt_pyk[2] # remove kalman gain that is not returned py _filter

filt =  list(map(lambda x: torch.stack(x), _filter(*args_filt_torch.values(), obs_mask=torch.tensor(obs_mask))))

test_close( filt_pyk, filt)

#### KalmanFilter method

In [ ]:
#| export
@patch
def _filter_all(self: KalmanFilter, obs, mask=None, check_args=None) -> Tuple:
    obs, obs_mask = self._parse_obs(obs, mask)

    return _filter(
            self.transition_matrices,
            self.obs_matrices,
            self.transition_cov,
            self.obs_cov,
            self.transition_offsets,
            self.obs_offsets,
            self.initial_state_mean,
            self.initial_state_cov,
            obs,
            obs_mask,
            check_args
        )

@patch
def filter(self: KalmanFilter,
          obs: Tensor, # [n_timesteps, n_dim_obs] obs for times [0...n_timesteps-1]
          mask = None,
          check_args=None
          ) -> ListNormal: # Filtered state
    """Filter observation"""
    _, _, filt_state_means, filt_state_covs = self._filter_all(obs, mask, check_args)
    # need to convert a list of tensors with gradients to a big tensors without gradients
    return ListNormal(_stack_detach(filt_state_means), _stack_detach(filt_state_covs))


#### Final Testing

due to numerical issue the error is bigger thatn `1e-5` which is default

In [ ]:
((tst.filter.filter(tst.data, tst.mask_torch)[1] -
np.float32(tst.filter_pyk.filter(tst.data_pyk)[1]))).max()

tensor(0.0164794922)

In [ ]:
tst = KalmanFilterTester()

In [ ]:
test_close(
    tst.filter.filter(tst.data, tst.mask_torch),
    tst.filter_pyk.filter(tst.data_pyk), eps=4e-2) # need to increase the resolution

using `float64` the error is smaller, which confirms that it is only a numerical issue

In [ ]:
test_close(
    tst64.filter.filter(tst.data, tst.mask_torch),
    tst64.filter_pyk.filter(tst.data_pyk))

In [ ]:
((tst64.filter.filter(tst64.data, tst64.mask_torch)[1] -
tst64.filter_pyk.filter(tst64.data_pyk)[1])).max()

tensor(4.6659886710e-07, dtype=torch.float64)

##### Old Testing

In [ ]:
test_close(
    pyk.filter(X.numpy()),
    k.filter(X)
)

In [ ]:
test_close(
    pyk2.filter(nX2),
    k2.filter(X2)
)

In [ ]:
k2.filter(X2m)

ListNormal(mean=tensor([[ 0.0000000000,  0.0000000000],
        [ 1.3333332539, 13.3333320618],
        [ 1.3333332539, 13.3333320618],
        [ 3.2727272511, 32.7272720337]]), cov=tensor([[[1.0000001192, 0.0000000000],
         [0.0000000000, 1.0000001192]],

        [[0.6666668653, 0.0000000000],
         [0.0000000000, 0.6666668653]],

        [[1.6666669846, 0.0000000000],
         [0.0000000000, 1.6666669846]],

        [[0.7272727489, 0.0000000000],
         [0.0000000000, 0.7272727489]]]))

In [ ]:
test_close(
    pyk.filter(X.numpy()),
    k.filter(X)
)

In [ ]:
test_close(
    pyk2.filter(nX2),
    k2.filter(X2)
)

In [ ]:
test_close(
    pyk2.filter(nX2m),
    k2.filter(X2m)
)

### Smooth

#### Smooth step

In [ ]:
#| export
def _smooth_update(transition_matrix,      # [n_dim_state, n_dim_state]
                   filt_state: Normal, # [n_dim_state] filtered state at time `t`
                   pred_state: Normal,        # [n_dim_state] state before filtering at time `t + 1` (= using the observation until time t)
                   next_smoothed_state: Normal, # [n_dim_state] smoothed state at time  `t+1`
                   check_args: dict|None = None # if not None checks that the result is positive definite
                   ) -> Normal: # mean and cov of smoothed state at time `t`
    r"""Correct a pred state with a Kalman Smoother update

    Calculates posterior distribution of the hidden state at time `t` given the the observations via Kalman Smoothing.
    """
    kalman_smoothing_gain = filt_state.cov @ transition_matrix.T @ torch.inverse(pred_state.cov)

    smoothed_state_mean = filt_state.mean + kalman_smoothing_gain @ (next_smoothed_state.mean - pred_state.mean)
    smoothed_state_cov = (filt_state.cov
                      + kalman_smoothing_gain @ (next_smoothed_state.cov - pred_state.cov) @ kalman_smoothing_gain.T)

    if check_args is not None: check_posdef(pred_state_cov, 'filter_correct', **check_args)
    
    return ListNormal(smoothed_state_mean, smoothed_state_cov,)

In [ ]:
args_np_sm = {
    'pred_state_m': np.zeros(2),
    'pred_state_cov': np.eye(2),
    'filt_state_m': np.zeros(2),
    'filt_state_cov': np.eye(2),
    'next_state_m': np.zeros(2),
    'next_state_cov': np.eye(2),
    'trans_m': np.eye(2),
}

args_torch_sm = {k: torch.tensor(v) for k,v in args_np_sm.items()}

In [ ]:
pyk_mean, pyk_cov, _ = pykalman.standard._smooth_update(
    args_np_sm['trans_m'],
    args_np_sm['filt_state_m'],
    args_np_sm['filt_state_cov'],
    args_np_sm['pred_state_m'],
    args_np_sm['pred_state_cov'],
    args_np_sm['next_state_m'],
    args_np_sm['next_state_cov'],
)

In [ ]:
torch_k = _smooth_update(
    args_torch_sm['trans_m'],
    Normal(args_torch_sm['filt_state_m'], args_torch_sm['filt_state_cov']),
    Normal(args_torch_sm['pred_state_m'], args_torch_sm['pred_state_cov']),
    Normal(args_torch_sm['next_state_m'], args_torch_sm['next_state_cov']),
)

In [ ]:
test_close((pyk_mean, pyk_cov, ), torch_k)

In [ ]:
#| export
def _smooth(transition_matrices, # `[n_timesteps-1, n_dim_state, n_dim_state]` or `[n_dim_state, n_dim_state]`
            filt_state: ListNormal, # `[n_timesteps, n_dim_state]`
                # `filt_state_means[t]` = mean state estimate for time t given obs from times `[0...t]`
            pred_state: ListNormal, # `[n_timesteps, n_dim_state]`
                # `pred_state_means[t]` = mean state estimate for time t given obs from times `[0...t-1]`
           check_args: dict|None = None # if not None checks that the result is positive definite
           ) -> ListNormal: # `[n_timesteps, n_dim_state]` Smoothed state 
    """Apply the Kalman Smoother """
    n_timesteps, n_dim_state = len(pred_state.mean), pred_state.mean[0].shape[0]

    smoothed_state = ListNormal(torch.zeros((n_timesteps,n_dim_state), dtype=pred_state.mean[0].dtype, device=pred_state.mean[0].device), 
                                torch.zeros((n_timesteps, n_dim_state,
                                           n_dim_state), dtype=pred_state.mean[0].dtype, device=pred_state.mean[0].device))

    smoothed_state.mean[-1] = filt_state.mean[-1]
    smoothed_state.cov[-1] = filt_state.cov[-1]

    for t in reversed(range(n_timesteps - 1)):
        transition_matrix = _last_dims(transition_matrices, t)
        (smoothed_state.mean[t], smoothed_state.cov[t]) = (
            _smooth_update(
                transition_matrix,
                filt_state.get_nth(t),
                pred_state.get_nth(t + 1),
                smoothed_state.get_nth(t+1),
                check_args = {'t': t, **check_args} if check_args is not None else None
            )
        )
    return smoothed_state

In [ ]:
(pred_state_means, pred_state_covs, filt_state_means, filt_state_covs ) = k2._filter_all(X2m)

In [ ]:
torch_smooth = _smooth(k2.transition_matrices,  ListNormal(filt_state_means, filt_state_covs), ListNormal(pred_state_means, pred_state_covs))

In [ ]:
pyk_sm_mean, pyk_sm_cov, _ = pykalman.standard._smooth(k2.transition_matrices.detach().numpy(),
                          _stack_detach(filt_state_means).numpy(), _stack_detach(filt_state_covs).numpy(),
                          _stack_detach(pred_state_means).numpy(), _stack_detach(pred_state_covs).numpy())

In [ ]:
test_close((pyk_sm_mean, pyk_sm_cov,), torch_smooth)

In [ ]:
torch_smooth

ListNormal(mean=tensor([[ 0.9090909362,  9.0909090042],
        [ 1.8181818724, 18.1818180084],
        [ 2.5454545021, 25.4545459747],
        [ 3.2727272511, 32.7272720337]], grad_fn=<CopySlices>), cov=tensor([[[0.6363637447, 0.0000000000],
         [0.0000000000, 0.6363637447]],

        [[0.5454546809, 0.0000000000],
         [0.0000000000, 0.5454546809]],

        [[0.9090909958, 0.0000000000],
         [0.0000000000, 0.9090909958]],

        [[0.7272727489, 0.0000000000],
         [0.0000000000, 0.7272727489]]], grad_fn=<CopySlices>))

#### KalmanFilter method

In [ ]:
#| export
@patch
def smooth(self: KalmanFilter,
           obs: Tensor, # dataset
           mask = None,
           check_args=None
          ) -> Tuple[Tensor, Tensor]:
                # `smoothed_state_means` : `[n_timesteps, n_dim_state]`
                    # mean of hidden state distributions for times `[0...n_timesteps-1]`
                # `smoothed_state_covs` : `[n_timesteps, n_dim_state]`
                    # covs of hidden state distributions for times `[0...n_timesteps-1]`
        
    """Smoothing for prediction (no gradients)"""

    (pred_state_means, pred_state_covs, filt_state_means, filt_state_covs) = self._filter_all(obs, mask, check_args)

    return _smooth(
            self.transition_matrices,
            ListNormal(filt_state_means, filt_state_covs),
            ListNormal(pred_state_means, pred_state_covs),
            check_args
        )

  

In [ ]:
k.smooth(X)

ListNormal(mean=tensor([[0.9230769277],
        [1.7692308426],
        [2.3846154213]], grad_fn=<CopySlices>), cov=tensor([[[0.3846154213]],

        [[0.4615385532]],

        [[0.6153846979]]], grad_fn=<CopySlices>))

#### Final Testing

In [ ]:
test_close(
    tst.filter.smooth(tst.data, tst.mask_torch),
    tst.filter_pyk.smooth(tst.data_pyk), eps=4e-2) # need to increase the resolution

In [ ]:
test_close(
    tst64.filter.smooth(tst64.data, tst64.mask_torch),
    tst64.filter_pyk.smooth(tst64.data_pyk))

In [ ]:
test_close(
    pyk.smooth(nX),
    k.smooth(X).detach()
)

In [ ]:
test_close(
    pyk2.smooth(nX2),
    k2.smooth(X2).detach()
)

In [ ]:
test_close(
    pyk2.smooth(nX2m),
    k2.smooth(X2m)
)

### Predict

In [ ]:
#| export
@patch
def _obs_from_state(self: KalmanFilter, state_mean, state_cov, check_args=None):

    mean = self.obs_matrices @ state_mean
    cov = self.obs_matrices @ state_cov @ self.obs_matrices.mT + self.obs_cov
    
    
    if check_args is not None: check_posdef(pred_state_cov, 'predict',  **check_args)
                
    
    return ListNormal(mean, cov)

@patch
def predict(self: KalmanFilter, obs, times, mask=None, smooth=True, check_args=None):
    state = self.smooth(obs, mask, check_args) if smooth else self.filter(obs, mask, check_args)
    times = array1d(times)
    
    n_timesteps = obs.shape[0]
    n_features = obs.shape[1] if len(obs.shape) > 1 else 1
    
    if times.max() > n_timesteps or times.min() < 0:
        raise ValueError(f"provided times range from {times.min()} to {times.max()}, which is outside allowed range : 0 to {n_timesteps}")

    means = torch.empty((times.shape[0], n_features), dtype=obs.dtype, device=obs.device)
    covs = torch.empty((times.shape[0], n_features, n_features), dtype=obs.dtype, device=obs.device) 
    for i, t in enumerate(times):
        mean, cov = self._obs_from_state(
            state.mean[t],
            state.cov[t],
            {'t': t, **check_args} if check_args is not None else None
        )
        
        means[i] = mean
        covs[i] = cov
    
    
    return ListNormal(means, covs)  

In [ ]:
k.predict(X, [1,2])

ListNormal(mean=tensor([[1],
        [2]]), cov=tensor([[[1]],

        [[1]]]))

pykalman doesn't support a predict method

In [ ]:
tst.filter.predict(tst.data, times = torch.tensor([0,1]), mask = tst.mask_torch)

ListNormal(mean=tensor([[-0.6047446132, -0.3410776258, -0.8897568583],
        [-0.6956433058, -0.1774560809, -0.2678230107]], grad_fn=<CopySlices>), cov=tensor([[[1.3786063194, 1.2001909018, 0.7694454193],
         [1.2001911402, 1.5085724592, 1.0023529530],
         [0.7694455385, 1.0023529530, 0.6710571051]],

        [[1.5426188707, 1.2861547470, 0.9304560423],
         [1.2861535549, 1.5494836569, 1.0789959431],
         [0.9304523468, 1.0789948702, 0.8079837561]]], grad_fn=<CopySlices>))

In [ ]:
mean, cov = tst64.filter.smooth(tst64.data, tst64.mask_torch)

In [ ]:
tst64.filter._obs_from_state(mean[0], cov[0])

ListNormal(mean=tensor([0.3861103695, 0.7449462019, 0.1333758789], dtype=torch.float64,
       grad_fn=<MvBackward0>), cov=tensor([[1.1592958931, 0.6811550132, 0.9250637696],
        [0.6811550132, 1.0738654742, 0.5473706487],
        [0.9250637696, 0.5473706487, 1.1431980715]], dtype=torch.float64,
       grad_fn=<AddBackward0>))

### Log Likelihood

In [ ]:
pykalman.standard.KalmanFilter.loglikelihood??

Signature: pykalman.standard.KalmanFilter.loglikelihood(self, X)
Source:   
    def loglikelihood(self, X):
        """Calculate the log likelihood of all observations

        Parameters
        ----------
        X : [n_timesteps, n_dim_obs] array
            observations for time steps [0...n_timesteps-1]

        Returns
        -------
        likelihood : float
            likelihood of all observations
        """
        Z = np.array(self._parse_observations(X))

        # initialize parameters
        (transition_matrices, transition_offsets,
         transition_covariance, observation_matrices,
         observation_offsets, observation_covariance,
         initial_state_mean, initial_state_covariance) = (
            self._initialize_parameters()
        )

        # apply the Kalman Filter
        (predicted_state_means, predicted_state_covariances,
         kalman_gains, filtered_state_means,
         filtered_state_covariances) = (
            _filter(
                tran

In [ ]:
pykalman.standard._loglikelihoods??

Signature:
pykalman.standard._loglikelihoods(
    observation_matrices,
    observation_offsets,
    observation_covariance,
    predicted_state_means,
    predicted_state_covariances,
    observations,
)
Source:   
def _loglikelihoods(observation_matrices, observation_offsets,
                    observation_covariance, predicted_state_means,
                    predicted_state_covariances, observations):
    """Calculate log likelihood of all observations

    Parameters
    ----------
    observation_matrices : [n_timesteps, n_dim_obs, n_dim_obs] or [n_dim_obs,
    n_dim_state] array
        observation matrices for t in [0...n_timesteps-1]
    observation_offsets : [n_timesteps, n_dim_obs] or [n_dim_obs] array
        offsets for observations for t = [0...n_timesteps-1]
    observation_covariance : [n_dim_obs, n_dim_obs] array
        covariance matrix for all observations
    predicted_state_means : [n_timesteps, n_dim_state] array
        mean of state at time t given observation

In [ ]:
#| export
@patch
def filter_loglikelihood(self: KalmanFilter, obs, mask=None):
    "Compute log likelihood using only filter step"
    # Those are the means and covs before the updating step,
    # otherwise the model would have already seen the observation that we are predicting 
    pred_state_mean, pred_state_cov, _, _ = self._filter_all(obs, mask)
    obs, obs_mask = self._parse_obs(obs, mask)

    max_t = obs.shape[0]
    lls = torch.zeros(max_t)
    for t in range(max_t):
        if obs_mask[t]:
            pred_obs_mean, pred_obs_cov = self._obs_from_state(pred_state_mean[t], pred_state_cov[t])
            ll = MultivariateNormal(pred_obs_mean, pred_obs_cov, validate_args=False).log_prob(obs[t])
            lls[t] = ll

    return lls.sum()

In [ ]:
k.filter_loglikelihood(X)

tensor(-5.2315979004, grad_fn=<SumBackward0>)

In [ ]:
test_close(k.filter_loglikelihood(X), pyk.loglikelihood(nX))

In [ ]:
pred_state, pred_state_cov, _, _ = tuple(map(_stack_detach, tst.filter._filter_all(tst.data, tst.mask_torch)))

In [ ]:
tst = KalmanFilterTester(nan_mask = False, p_missing=0)

In [ ]:
pykalman.standard._loglikelihoods(
    tst.params_pyk['observation_matrices'],
    tst.params_pyk['observation_offsets'],
    tst.params_pyk['observation_covariance'],
    pred_state.numpy(),
    pred_state_cov.numpy(),
    np.array(tst.data_pyk)
)

In [ ]:
test_close(
    tst.filter.filter_loglikelihood(tst.data, tst.mask_torch),
    tst.filter_pyk.loglikelihood(tst.data_pyk)) # need to increase the resolution

In [ ]:
k2.filter_loglikelihood(X2)

In [ ]:
test_close(k2.filter_loglikelihood(X2), pyk2.loglikelihood(nX2), eps=1e-4)

since the goal is to fill gaps we want the log likelihood for the whole gap and only for it

In [ ]:
#| export
@patch
def loglikelihood(self: KalmanFilter,
                  obs_train: Tensor, # [n_timesteps, n_dim_obs] Observations use for the filter (can containt missing data)
                  times: Tensor, # [n_pred_timesteps] time at which to calculate the log likelihood
                  obs_test: Tensor # [n_pred_timesteps, n_dim_obs] observed data to compute log likelihood
                 ) -> Tensor: # scalar that is sum of log likelihoods for all `times`
    "Log likelihood only for the `obs_test` at giben times"
    means, covs = self.predict(obs_train, times)
    lls = torch.zeros(len(times))
    for t in range(len(times)):
        lls[t] = MultivariateNormal(means[t], covs[t], validate_args=False).log_prob(obs_test[t:t+1])
    return lls.sum() 
        

In [ ]:
k.loglikelihood(X, [1,2], X[[1,2]])

In [ ]:
k2.loglikelihood(X2, [1,2], X2[[1,2]])

In [ ]:
k2.loglikelihood(X2m, [1,2], X2[[1,2]])

### Get Info

In [ ]:
#| export
@patch
def get_info(self: KalmanFilter, var_names=None):
    out = {}
    if var_names is not None: self.var_names = var_names 
    latent_names = [f"z_{i}" for i in range(self.transition_matrices.shape[0])]
    out['A'] = array2df(self.transition_matrices, latent_names, latent_names, 'latent')
    out['H'] = array2df(self.obs_matrices,        var_names,    latent_names, 'variable')
    out['R'] = array2df(self.obs_cov,             var_names,    var_names,     'variable')
    out['Q'] = array2df(self.transition_cov,      latent_names, latent_names, 'latent')
    return out

In [ ]:
display_as_row(k.get_info())

In [ ]:
display_as_row(k2.get_info())

## Train Parameters

This implementation of `KalmanFilter` allows to find the optimal parameters by maximising the log-likelihood using gradient descend

In [ ]:
training_iter = 200
k = KalmanFilter()
k.train()

optimizer = torch.optim.Adam(k.parameters(), lr=0.005) 

losses = []
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    loss = - k.loglikelihood(X, range(len(X)), X)
    losses.append(loss.item())
    # backpropagate gradients
    loss.backward()
    optimizer.step()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(losses)

In [ ]:
list(k.parameters())

## Other

### Testing

### Fuzzing smoother

trying to run the filter many times to see if some of the matrix are not symmetric

In [ ]:
from torch._C import _LinAlgError

In [ ]:
def fuzz_symmetric(n_iter=10, n_obs=100, **kwargs):
    tst = KalmanFilterTester(n_obs=n_obs, **kwargs)
    _, sm_covs = tst.filter.smooth(tst.data, tst.mask_torch)
    i_posdef = []
    for t, cov in enumerate(sm_covs):
        i_posdef.append(check_posdef(cov))
    return pd.concat(i_posdef)
    

In [ ]:
def fuzz_smooth(n_obs=100, **kwargs):
    tst = KalmanFilterTester(n_obs=n_obs, **kwargs)
    _, sm_covs = tst.filter.smooth(tst.data, tst.mask_torch, check_args= {})
    

In [ ]:
def find_max_obs(start=100_000, end=100, steps=10, **kwargs):
    for n in torch.logspace(torch.log10(start), torch.log10(end), 10):
        try:
            %time fuzz_smooth(n_obs=int(n), **kwargs)
            print(n, "working")
            break
        except _LinAlgError:
            print(n, "not working")
    

In [ ]:
import warnings

In [ ]:
posdef_log = pd.DataFrame()

In [ ]:
total_warn = []
for n in range(2, 200):
    with warnings.catch_warnings(record=True) as w:
        try:
            fuzz_smooth(n_obs=n)
        except _LinAlgError:
            print(n)
            break
        finally:
            total_warn.append((n, len(w)))
    

In [ ]:
posdef_log

In [ ]:
import altair as alt

In [ ]:
alt.Chart(pd.DataFrame(total_warn, columns=["n_obs", "n_not_posdef"])).mark_line().encode(alt.X("n_obs"), alt.Y("n_not_posdef"))

In [ ]:
alt.data_transformers.enable('data_server')

In [ ]:
alt.Chart(posdef_log).mark_line().encode(alt.X("t"), alt.Y("average(sym_upto)"))

In [ ]:
alt.Chart(posdef_log).mark_point().encode(alt.X("t"), alt.Y("count(is_sym)"))

In [ ]:
posdef_log[["t", "name"]]

In [ ]:
plt.scatter(posdef_log.reset_index().index, posdef_log.sym_upto)

In [ ]:
for i in range(3):
    %time 1+1

In [ ]:
# find_max_obs(dtype=torch.float64)

The function takes 5 min to run so this is the output saved

with `float64` there is no problem with positive definite matrices even with 100k observations
```CPU times: user 30min 43s, sys: 28.6 s, total: 31min 11s
Wall time: 5min 26s
tensor(100000.) working```

In [ ]:
tst = KalmanFilterTester(n_obs=100)

In [ ]:
is_posdef(tst.params['obs_cov'])

In [ ]:
is_posdef(tst.params['transition_cov'])

In [ ]:
is_posdef(tst.params['initial_state_cov'])

In [ ]:
tst.filter.smooth(tst.data, tst.mask_torch);

#### Random Testing

The goal is to generate random set of data and parameters and check that `meteo_imp` implementation is the same of `pykalman` implementation

In [ ]:
n_dim_state = 3 
n_dim_obs = 3
n_obs = 10
p_missing = .3

In [ ]:
to_posdef = PosDef().transform

In [ ]:
data = torch.rand(n_obs, n_dim_obs)
mask = torch.rand(n_obs, n_dim_obs) > p_missing
mask_torch = mask.all(1)

In [ ]:
mask[:10]

In [ ]:
mask_torch[:10]

In [ ]:
params = {
    'transition_matrices': torch.rand(n_dim_state, n_dim_state),
    'transition_offsets':  torch.rand(n_dim_state),        
    'transition_cov':      to_posdef(torch.rand(n_dim_state, n_dim_state)),        
    'obs_matrices':        torch.rand(n_dim_obs, n_dim_state),
    'obs_offsets':         torch.rand(n_dim_obs),          
    'obs_cov':             to_posdef(torch.rand(n_dim_obs, n_dim_obs)),            
    'initial_state_mean':  torch.rand(n_dim_state),        
    'initial_state_cov':   to_posdef(torch.rand(n_dim_state, n_dim_state)),
}

In [ ]:
params2pyk = {
    'transition_matrices': 'transition_matrices',
    'transition_offsets':  'transition_offsets',        
    'transition_cov':      'transition_covariance',        
    'obs_matrices':        'observation_matrices',
    'obs_offsets':         'observation_offsets',          
    'obs_cov':             'observation_covariance',            
    'initial_state_mean':  'initial_state_mean',        
    'initial_state_cov':   'initial_state_covariance',
}

In [ ]:
params

In [ ]:
k = KalmanFilter(**params)

In [ ]:
pred = k.smooth(data, mask_torch)

make a `pykalman` model using the same parameters

In [ ]:
data_pyk = np.ma.masked_array(data.numpy(), mask = mask.numpy())

In [ ]:
pyk_k = pykalman.standard.KalmanFilter(

    transition_matrices=k.transition_matrices.detach().numpy(),
    transition_offsets=k.transition_offsets.detach().numpy(),
    transition_covariance=k.transition_cov.detach().numpy(),
    observation_matrices=k.obs_matrices.detach().numpy(),
    observation_offsets=k.obs_offsets.detach().numpy(),
    observation_covariance=k.obs_cov.detach().numpy(),
    initial_state_mean=k.initial_state_mean.detach().numpy(),
    initial_state_covariance=k.initial_state_cov.detach().numpy()
)

In [ ]:
pred_pyk = pyk_k.smooth(data_pyk)

In [ ]:
for p in params.keys():
    print(p, getattr(k, p))

In [ ]:
for p in params.keys():
    print(p, getattr(pyk_k, params2pyk[p]))

### Compare Statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels

In [ ]:
# sm_kf = statsmodels.tsa.statespace.kalman_filter.KalmanFilter(
#     k_endog = 3,
#     k_states = 3,
#     initialization = 'known',
#     initial_state = pyk_ncov.initial_state_mean,
#     initial_state_cov = pyk_ncov.initial_state_covariance,
#     design = pyk_ncov.observation_matrices,
#     obs_cov = pyk_ncov.observation_covariance,
#     transition = pyk_ncov.transition_matrices,
#     state_cov = pyk_ncov.transition_covariance)

In [ ]:
# sm_kf.bind(X_ncov.detach().numpy())

In [ ]:
# sm_pred = sm_kf.filter() 

In [ ]:
# sm_pred.predicted_state.shape

In [ ]:
# sm_pred.predicted_state_cov.shape

In [ ]:
# mean = MultivariateNormal(torch.tensor(sm_pred.predicted_state[:, 0]), torch.tensor(sm_pred.predicted_state_cov[:, :, 0]))

In [ ]:
# sm_kf.loglikeobs()

## Export

In [ ]:
#| hide
from nbdev import nbdev_export
# nbdev_export()